# 5.11. Performing out-of-core computations on large arrays with Dask

In [ ]:
import numpy as np
import dask.array as da
import memory_profiler

In [ ]:
%load_ext memory_profiler

In [ ]:
Y = da.random.normal(size=(10000, 10000),
                     chunks=(1000, 1000))

In [ ]:
Y

In [ ]:
Y.shape, Y.size, Y.chunks

In [ ]:
mu = Y.mean(axis=0)
mu

In [ ]:
mu[0].compute()

In [ ]:
def f_numpy():
    X = np.random.normal(size=(10000, 10000))
    x = X.mean(axis=0)[0:100]

In [ ]:
%%memit
f_numpy()

In [ ]:
%%time
f_numpy()

In [ ]:
def f_dask():
    Y = da.random.normal(size=(10000, 10000),
                         chunks=(1000, 1000))
    y = Y.mean(axis=0)[0:100].compute()

In [ ]:
%%memit
f_dask()

In [ ]:
%%time
f_dask()

In [ ]:
def f_dask2():
    Y = da.random.normal(size=(10000, 10000),
                         chunks=(10000, 100))
    y = Y.mean(axis=0)[0:100].compute()

In [ ]:
%%memit
f_dask2()

In [ ]:
%%time
f_dask2()

In [ ]:
from dask.distributed import Client

In [ ]:
client = Client()

In [ ]:
client

In [ ]:
Y.sum().compute()

In [ ]:
future = client.compute(Y.sum())

In [ ]:
future

In [ ]:
future.result()

In [ ]:
huge = da.random.uniform(
    size=(1500000, 100000), chunks=(10000, 10000))
"Size in memory: %.1f GB" % (huge.nbytes / 1024 ** 3)

In [ ]:
from dask.diagnostics import ProgressBar
# WARNING: this will take a very long time computing
# useless values. This is for pedagogical purposes
# only.
with ProgressBar():
    m = huge.mean().compute()

## Cleanup

In [ ]:
!rm -rf dask-worker-space